# Mod 3 Final Project

## Student Info

- Name: Susanna Han
- Cohort: Part Time
- Instructor: James


## Instructions:

- Open and read the project assignment and guidelines in `MOD_PROJECT_README.ipynb`
- Review the hypothesis testing workflow found in this repo's `README.md` and inside `hypothesis_testing_workflow.ipynb` (also at the bottom of the `MOD_PROJECT_README.ipynb`)

- 3 functions from study group/learn.co lessons have been provided inside `functions.py`
    - `Cohen_d`, `find_outliers_IQR`,`find_outliers_Z`

<img src="https://raw.githubusercontent.com/jirvingphd/dsc-mod-3-project-online-ds-ft-100719/master/Northwind_ERD_updated.png">

# PROJECT

In [ ]:
!pip install -U fsds_100719
from fsds_100719.imports import *

import pandas as pd

In [ ]:
import pandas as pd
import scipy.stats as stats
import pandas as pd
import numpy as np

In [ ]:
def Cohen_d(group1, group2, correction = False):
    """Compute Cohen's d
    d = (group1.mean()-group2.mean())/pool_variance.
    pooled_variance= (n1 * var1 + n2 * var2) / (n1 + n2)

    Args:
        group1 (Series or NumPy array): group 1 for calculating d
        group2 (Series or NumPy array): group 2 for calculating d
        correction (bool): Apply equation correction if N<50. Default is False. 
            - Url with small ncorrection equation: 
                - https://www.statisticshowto.datasciencecentral.com/cohens-d/ 
    Returns:
        d (float): calculated d value
         
    INTERPRETATION OF COHEN's D: 
    > Small effect = 0.2
    > Medium Effect = 0.5
    > Large Effect = 0.8
    
    """
    import scipy.stats as stats
    import scipy   
    import numpy as np
    N = len(group1)+len(group2)
    diff = group1.mean() - group2.mean()

    n1, n2 = len(group1), len(group2)
    var1 = group1.var()
    var2 = group2.var()

    # Calculate the pooled threshold as shown earlier
    pooled_var = (n1 * var1 + n2 * var2) / (n1 + n2)
    
    # Calculate Cohen's d statistic
    d = diff / np.sqrt(pooled_var)
    
    ## Apply correction if needed
    if (N < 50) & (correction==True):
        d=d * ((N-3)/(N-2.25))*np.sqrt((N-2)/N)
    return d


#Your code here
def find_outliers_z(data):
    """Use scipy to calculate absolute Z-scores 
    and return boolean series where True indicates it is an outlier.

    Args:
        data (Series,or ndarray): data to test for outliers.

    Returns:
        [boolean Series]: A True/False for each row use to slice outliers.
        
    EXAMPLE USE: 
    >> idx_outs = find_outliers_df(df['AdjustedCompensation'])
    >> good_data = df[~idx_outs].copy()
    """
    import pandas as pd
    import numpy as np
    import scipy.stats as stats
    import pandas as pd
    import numpy as np
    ## Calculate z-scores
    zs = stats.zscore(data)
    
    ## Find z-scores >3 awayfrom mean
    idx_outs = np.abs(zs)>3
    
    ## If input was a series, make idx_outs index match
    if isinstance(data,pd.Series):
        return pd.Series(idx_outs,index=data.index)
    else:
        return pd.Series(idx_outs)
    
    
    
def find_outliers_IQR(data):
    """Use Tukey's Method of outlier removal AKA InterQuartile-Range Rule
    and return boolean series where True indicates it is an outlier.
    - Calculates the range between the 75% and 25% quartiles
    - Outliers fall outside upper and lower limits, using a treshold of  1.5*IQR the 75% and 25% quartiles.

    IQR Range Calculation:    
        res = df.describe()
        IQR = res['75%'] -  res['25%']
        lower_limit = res['25%'] - 1.5*IQR
        upper_limit = res['75%'] + 1.5*IQR

    Args:
        data (Series,or ndarray): data to test for outliers.

    Returns:
        [boolean Series]: A True/False for each row use to slice outliers.
        
    EXAMPLE USE: 
    >> idx_outs = find_outliers_df(df['AdjustedCompensation'])
    >> good_data = df[~idx_outs].copy()
    
    """
    df_b=data
    res= df_b.describe()

    IQR = res['75%'] -  res['25%']
    lower_limit = res['25%'] - 1.5*IQR
    upper_limit = res['75%'] + 1.5*IQR

    idx_outs = (df_b>upper_limit) | (df_b<lower_limit)

    return idx_outs



def prep_data_for_tukeys(data):
    """Accepts a dictionary with group names as the keys 
    and pandas series as the values. 
    
    Returns a dataframe ready for tukeys test:
    - with a 'data' column and a 'group' column for sms.stats.multicomp.pairwise_tukeyhsd 
    
    Example Use:
    df_tukey = prep_data_for_tukeys(grp_data)
    tukey = sms.stats.multicomp.pairwise_tukeyhsd(df_tukey['data'], df_tukey['group'])
    tukey.summary()
    """
    
    df_tukey = pd.DataFrame(columns=['data','group'])
    for k,v in  data.items():
        grp_df = v.rename('data').to_frame() 
        grp_df['group'] = k
        df_tukey=pd.concat([df_tukey, grp_df],axis=0)

	## New lines added to ensure compatibility with tukey's test
    df_tukey['group'] = df_tukey['group'].astype('str')
    df_tukey['data'] = df_tukey['data'].astype('float')
    return df_tukey


In [ ]:
import sqlite3
connect = sqlite3.connect('Northwind_small.sqlite')
cur = connect.cursor()

In [ ]:
## To see all tables
cur.execute("""SELECT name FROM sqlite_master WHERE type='table';""")
df_tables = pd.DataFrame(cur.fetchall(), columns=['Table'])
df_tables

# Hypothesis 1

> Does discount amount have a statistically significant effect on the quantity of a product in an order? If so, at what level(s) of discount?

- $H_0$: The customer orders the same quantity of a product whether it's discounted or full price. 
- $H_A$: The customer orders more or less of a product whether it's discounted or full price.

In [ ]:
cur.execute('''SELECT * from OrderDetail''')
col_names = [x[0] for x in cur.description]
col_names

In [ ]:
df = pd.DataFrame(cur.fetchall(), columns = col_names)

In [ ]:
df['Discount'] = df['Discount'] > 0
df['Discount'] = df['Discount'].map({True:'Discounted', False: 'Full Price'})
df

In [ ]:
sns.barplot(data=df, x = df['Discount'], y= df['Quantity'], ci=68)

In [ ]:
discounts = {}
for discount_grps in df['Discount'].unique():
    discounts[discount_grps]= df.groupby('Discount').get_group(discount_grps)['Quantity']
discounts

In [ ]:
fig, ax=plt.subplots(figsize = (8,5))
for discount_grps, grp_data in discounts.items():
    sns.distplot(grp_data, label=discount_grps, ax=ax)
ax.legend()
ax.set(title ='Quantity by Discounted Group', ylabel = 'Density')

In [ ]:
for discount_grps, grp_data in discounts.items():
    idx_outs = find_outliers_z(grp_data)
    print(f'[I] found {idx_outs.sum()} outliers in {discount_grps} using z-scores')
    discounts[discount_grps] = grp_data[-idx_outs]

In [ ]:
fig, ax=plt.subplots(figsize = (8,5))
for discount_grps, grp_data in discounts.items():
    sns.distplot(grp_data, label=discount_grps, ax=ax )
ax.legend()
ax.set(title ='Quantity by Discounted Group', ylabel = 'Density')

In [ ]:
for k,v in discounts.items():
    stat,p = stats.normaltest(v)
    print(f"The {k} Normaltest p-value = {round(p,4)}")

In [ ]:
print(f"There are {len(discounts['Full Price'])} full price items.")
print(f"There are {len(discounts['Discounted'])} discounted items.")

In [ ]:
#beause the p-value is < 0.05 the data is not normal but since the size of our data is big enough we can ignore the results of the normal test.
#Now we will test for equal variance by using the levene's test.

In [ ]:
data = []
for k,v in discounts.items():
    data.append(v)
data[0]

In [ ]:
stats.levene(*data) 

# Hypothesis 2

> Does discount have a statistically significant effect on the total amount spent in an order? 

- $H_0$: The customer spends the same amount of money whether it's discounted or full price. 
- $H_A$: The customer spends more or less money whether it's discounted or full price.

In [ ]:
df['Total Spent'] = df['UnitPrice'] * df['Quantity']
df

In [ ]:
sns.barplot(data=df, x = df['Discount'], y= df['Total Spent'], ci=68)

In [ ]:
#Looking at the bar graph we can see that the discounted products have a larger spending amount than the full priced items.

In [ ]:
discounts = {}
for discount_grps in df['Discount'].unique():
    discounts[discount_grps]= df.groupby('Discount').get_group(discount_grps)['Total Spent']
discounts

In [ ]:
fig, ax=plt.subplots(figsize = (8,5))
for discount_grps, grp_data in discounts.items():
    sns.distplot(grp_data, label=discount_grps, ax=ax )
ax.legend()
ax.set(title ='Total Spent by Discounted Group', ylabel = 'Density')

In [ ]:
for discount_grps, grp_data in discounts.items():
    idx_outs = find_outliers_z(grp_data)
    print(f'[I] found {idx_outs.sum()} outliers in {discount_grps} using z-scores')
    discounts[discount_grps] = grp_data[-idx_outs]

In [ ]:
fig, ax=plt.subplots(figsize = (8,5))
for discount_grps, grp_data in discounts.items():
    sns.distplot(grp_data, label=discount_grps, ax=ax )
ax.legend()
ax.set(title ='Total Spent by Discounted Group', ylabel = 'Density')

In [ ]:
for k,v in discounts.items():
    stat,p = stats.normaltest(v)
    print(f"The {k} Normaltest p-value = {round(p,4)}")

In [ ]:
print(f"There are {len(discounts['Full Price'])} full price items.")
print(f"There are {len(discounts['Discounted'])} discounted items.")

In [ ]:
data = []
for k,v in discounts.items():
    data.append(v)
data[0]

In [ ]:
stats,p = stats.levene(*data) 
print(f'Levenes test for equal variance p-value ={round(p,4)} therefore it does not have equal variance')

# Hypothesis 3

> Does the supplier region have a statistically significant effect on the product quantity sold.

- $H_0$: The supplier region does not have an effect on the product quantity sold.
- $H_A$: THe supplier region does have an effect on the product quantity sold.

In [ ]:
cur.execute('''SELECT *
               FROM Supplier
               JOIN Product
               JOIN OrderDetail;''')
df = pd.DataFrame(cur.fetchall()) 
col_names = [x[0] for x in cur.description]
df.head()

In [ ]:
df = pd.DataFrame(cur.fetchall(), columns = col_names)

In [ ]:
discounts = {}
for discount_grps in df['Discount'].unique():
    discounts[discount_grps]= df.groupby('Discount').get_group(discount_grps)['Quantity']
discounts

In [ ]:
df['Region'].unique()

In [ ]:
sns.barplot(data=df, x = df['Region'], y= df['Quantity'], ci=68)
region - product id - Quantity

# Hypothesis 4

> Does the customer region have a statistically significant effect on the number of sales?

- $H_0$:
- $H_A$: